In [1]:
import os
import sys

# Get the absolute path of the notebook's directory
notebook_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.append(notebook_dir)

In [2]:
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
from src.app import PermutationSolver


In [12]:
def factorial(n):
    if n == 0 or n == 1:
        return 1
    else:
        return n * factorial(n - 1)

# Example usage
n=12
n_factorial = factorial(n)  # Change 5 to any number you want to calculate the factorial for
print(f"{n}! = {n_factorial}")


12! = 479001600


In [3]:
def factorial(n):
    if n == 0 or n == 1:
        return 1
    else:
        return n * factorial(n - 1)

# Example usage
n=16
n_factorial = factorial(n)  # Change 5 to any number you want to calculate the factorial for
print(f"{n}! = {n_factorial}")


16! = 20922789888000


In [8]:
def factorial(n):
    if n == 0 or n == 1:
        return 1
    else:
        return n * factorial(n - 1)

# Example usage
n=100
n_factorial = factorial(n)  # Change 5 to any number you want to calculate the factorial for
print(f"{n}! = {n_factorial}")


100! = 93326215443944152681699238856266700490715968264381621468592963895217599993229915608941463976156518286253697920827223758251185210916864000000000000000000000000


In [4]:
n = 16
n * (n-1) / 2

120.0

In [5]:
# n_perm_list = [9]
# n_epoch_list = [200]
# n_epoch_dqn_list = [15_000]
# batch_size_list = [2**10, 2**14]
# lr_supervised_list = [1e-3]
# lr_rl_list = [1e-3, 5e-4, 1e-4]
# n_rnd_walks_sup = [100_000]
# n_rnd_walks_rl = [10_000, 100_000]
# anchor_list = [0, 1]
# beam_width_list = [2**0]
# list_layers_sizes = [2**12, 2**14]

n_perm_list = [9]
n_epoch_list = [1000]
n_epoch_dqn_list = [15_000]
batch_size_list = [2**10]
lr_supervised_list = [1e-3]
lr_rl_list = [5e-1]
n_rnd_walks_sup = [1_000]
n_rnd_walks_rl = [1_000]
anchor_list = [1]
beam_width_list = [2**0]
list_layers_sizes = [2**9]



summary_df = pd.DataFrame()

for n_permutations_length in n_perm_list:

    n = n_permutations_length
    config = {
                'n_permutations_length': n,
                
                # Random walks params
                'random_walks_type': 'non-backtracking-beam',
                'n_random_walk_length': int(n * (n-1) / 2),
                'n_random_walks_to_generate': 1000,
                'n_random_walks_steps_back_to_ban': 8,
                
                # Neural Net params
                'model_type': 'MLP',
                'list_layers_sizes': [4096],
                'n_epochs': 100,
                'batch_size': 1024,
                'lr_supervised': 1e-3,
                
                # DQN training
                'n_epochs_dqn': 5000,
                'flag_dqn_round': False,
                'n_random_walks_to_generate_dqn': 1000,
                'verbose_loc': 50,
                'lr_rl': 1e-3,
                
                # Beam search
                'beam_search_torch': True,
                'beam_search_Fironov': False,
                'beam_width': 1,
                'n_steps_limit': 4 * n**2,
                'alpha_previous_cost_accumulation': 0,
                'beam_search_models_or_heuristics': 'model_torch',
                'ban_p0_p1_transposition_if_p0_lt_p1_ie_already_sorted': False,
                'n_beam_search_steps_back_to_ban': 32,
                
                # What to solve
                'solve_random_or_longest_state': 'solve_LRX_longest',
                'verbose': 100
            }

    config['mode'] = 'single_hard_hinge'
    config['w_hinge'] = 1.0
    
    for anchor_mode in anchor_list:
        config['w_anchor'] = anchor_mode
        for n_rnd_walks_sup_mode in n_rnd_walks_sup:
            config['n_random_walks_to_generate'] = n_rnd_walks_sup_mode
            for n_rnd_walks_rl_mode in n_rnd_walks_rl:
                config['n_random_walks_to_generate_dqn'] = n_rnd_walks_rl_mode
                solver = PermutationSolver(config)
                solver.config['n_permutations_length'] = n_permutations_length
                for list_layers_size in list_layers_sizes:
                    solver.config['list_layers_sizes'] = [list_layers_size]
                    print(list_layers_size)
                    for lr in lr_supervised_list:
                        solver.config['lr_supervised'] = lr
                        for lr_rl in lr_rl_list:
                            solver.config['lr_rl'] = lr_rl
                            for batch_size in batch_size_list:
                                solver.config['batch_size'] = batch_size
                                for n_epoch in n_epoch_list:
                                    solver.config['n_epochs'] = n_epoch
                                    mlp_losses = solver.train_mlp(mode = 'anchor')
                                    # save mlp_model
                                    torch.save(solver.mlp_model.state_dict(), f'models/mlp_model_{n_permutations_length}_{n_epoch}_{list_layers_size}_{lr}_{batch_size}.pth')
                                    for n_epoch_dqn in n_epoch_dqn_list:
                                        solver.config['n_epochs_dqn'] = n_epoch_dqn
                                        dqn_losses = solver.train_dqn()
                                        # save dqn_model
                                        torch.save(solver.dqn_model.state_dict(), f'models/dqn_model_{n_permutations_length}_{n_epoch_dqn}_{list_layers_size}_{lr}_{batch_size}.pth')
                                        for beam_width in beam_width_list:
                                            solver.config['beam_width'] = beam_width
                                            i_step, flag_found_destination, path = solver.test_beam_search()
            
                                            # calculate min and max of difference between solver.y_anchor and y_valid
                                            # y_valid = solver.dqn_model(solver.X_anchor)
                                            device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
                                            solver.dqn_trainer.model.eval()               # set to eval mode
                                            y_parts = []
                                            n_samples = len(solver.X_anchor)
                                            with torch.no_grad():                 # <— no grad tracking
                                                for start in range(0, n_samples, batch_size):
                                                    end = start + batch_size
                                                    X_batch = solver.X_anchor[start:end].to(device)
                                                    y_batch = solver.dqn_trainer.model(X_batch)
                                                    y_parts.append(y_batch.cpu()) # move back to CPU
                                                    # explicitly free GPU memory
                                                    del X_batch, y_batch
                                                    torch.cuda.empty_cache()
                                            
                                            y_valid = torch.cat(y_parts, dim=0)
                                            
                                            diff = solver.y_anchor.cpu().detach().numpy() - y_valid.cpu().detach().numpy().reshape(1, -1)[0]
                                            min_diff = diff.min()
                                            max_diff = diff.max()
                                            std_diff = diff.std()
                                            mean_diff = diff.mean()
                                            num_elements_less_than_minus_05 = (diff < -0.5).sum()
                                            num_elements_larger_than_05 = (diff > 0.5).sum()
                                            wrong_predictions = num_elements_less_than_minus_05 + num_elements_larger_than_05
                                            percentage_wrong_predictions = wrong_predictions / len(diff)
            
                                            # save summary_df
                                            summary_df_cur = pd.DataFrame({'n_permutations_length': n_permutations_length,
                                                                           'n_rnd_walks_sup_mode': n_rnd_walks_sup_mode,
                                                                           'n_rnd_walks_rl_mode': n_rnd_walks_rl_mode,
                                                                           'anchor':anchor_mode,
                                                                           'batch_size':batch_size,
                                                                           'list_layers_sizes': [list_layers_size], 
                                                                           'lr_supervised': lr,
                                                                           'lr_rl': lr_rl, 
                                                                           'n_epoch': n_epoch, 
                                                                           'n_epoch_dqn': n_epoch_dqn, 
                                                                           'beam_width': beam_width, 
                                                                           'i_step': i_step, 
                                                                           'flag_found_destination': flag_found_destination, 
                                                                           'mlp_losses': mlp_losses[-1], 
                                                                           'dqn_losses': dqn_losses[-1],
                                                                           'min_diff': min_diff,
                                                                           'max_diff': max_diff,
                                                                           'mean_diff': mean_diff,
                                                                           'std_diff': std_diff,
                                                                           'num_elements_less_than_minus_05': num_elements_less_than_minus_05,
                                                                           'num_elements_larger_than_05': num_elements_larger_than_05,
                                                                           'percentage_wrong_predictions': percentage_wrong_predictions
                                                                           })
                                            summary_df = pd.concat([summary_df, summary_df_cur])
                                            summary_df.to_csv('models/summary_df.csv', index=False)

512
362880
X.shape: torch.Size([362880, 9])
y.shape: torch.Size([362880])


Training MLP with anchor:   1%|          | 10/1000 [00:05<09:26,  1.75it/s]

Epoch 10, Loss: 0.3234


Training MLP with anchor:   2%|▏         | 20/1000 [00:11<09:25,  1.73it/s]

Epoch 20, Loss: 0.1973


...

Epoch 5600 | Loss: 0.1006 | Hinge: 0.0181 | Anchor: 0.0682 | Times - RW: 0.08s, Bellman: 0.02s, Train: 0.06s


Training DQN:  38%|███▊      | 5652/15000 [15:01<24:53,  6.26it/s]

Epoch 5650 | Loss: 0.1487 | Hinge: 0.0158 | Anchor: 0.1286 | Times - RW: 0.07s, Bellman: 0.01s, Train: 0.07s


Training DQN:  38%|███▊      | 5676/15000 [15:05<24:47,  6.27it/s]


KeyboardInterrupt: 

In [4]:
solver.generate_training_data_anchor()

362880
X.shape: torch.Size([362880, 9])
y.shape: torch.Size([362880])


(tensor([[0, 1, 2,  ..., 6, 7, 8],
         [1, 2, 3,  ..., 7, 8, 0],
         [8, 0, 1,  ..., 5, 6, 7],
         ...,
         [0, 8, 7,  ..., 3, 2, 1],
         [2, 1, 0,  ..., 5, 4, 3],
         [1, 0, 8,  ..., 4, 3, 2]], device='cuda:0'),
 tensor([ 0,  1,  1,  ..., 35, 35, 36], device='cuda:0'))

In [ ]:
import matplotlib.pyplot as plt

# Plot distribution of the torch vector y_valid
plt.figure(figsize=(10, 6))
plt.hist(diff, bins=30, alpha=0.7, color='blue')
plt.title('Distribution of y_valid')
plt.xlabel('Value')
plt.ylabel('Frequency')
plt.grid(True)
plt.show()

In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import hsv_to_rgb
import ipywidgets as widgets
from IPython.display import display, clear_output


def visualize_tensor_sequence_notebook(tensor_list):
    """
    Интерактивная визуализация в Jupyter Notebook в виде линии шариков:
    - Использует ipywidgets.IntSlider и перерисовывает график в Output
    - tensor_list: список torch.Tensor размера (N, M)
    """
    M = tensor_list[0].shape[0]
    N = len(tensor_list)
    # позиции на линии: равномерные точки по оси X
    xs = np.arange(M)
    ys = np.zeros(M)

    # слайдер и область вывода
    slider = widgets.IntSlider(value=0, min=0, max=N-1, step=1, description='Frame')
    out = widgets.Output()
    
    def plot_frame(idx):
        vals = tensor_list[idx]
        arr = vals.cpu().numpy().astype(float)
        norm = arr / arr.max()
        hsv = np.stack([np.zeros_like(norm), norm, np.ones_like(norm)], axis=1)
        rgb = hsv_to_rgb(hsv)
        
        with out:
            clear_output(wait=True)
            fig, ax = plt.subplots(figsize=(8,2))
            ax.set_aspect('equal')
            ax.axis('off')
            # рисуем шарики вдоль линии
            scatter = ax.scatter(xs, ys, s=800, color=rgb)
            # цифры внутри шариков
            for x, y, v in zip(xs, ys, vals):
                ax.text(x, y, str(v.item()), ha='center', va='center', fontsize=12, color='black')
            # статус 'start' и 'done'
            status = 'start' if idx == 0 else ('done' if idx == N-1 else '')
            ax.text(M/2 - 0.5, 0.3, status, ha='center', va='bottom', fontsize=16)
            # установка границ
            ax.set_xlim(-1, M)
            ax.set_ylim(-1, 1)
            plt.show()

    # подписка на изменение слайдера
    slider.observe(lambda change: plot_frame(change['new']), names='value')
    display(slider, out)
    # отрисовка начального кадра
    plot_frame(0)


visualize_tensor_sequence_notebook(path)

In [ ]:
num_params = sum(p.numel() for p in solver.mlp_model.parameters())
print(f"Number of parameters in the model: {num_params}")